In [6]:
import pandas_datareader as pdr

In [7]:
import datetime

In [8]:
ticker = 'NFLX'

In [18]:
ohlcv = pdr.get_data_yahoo(ticker, datetime.date.today()- datetime.timedelta(1025), datetime.date.today())

In [19]:
ohlcv

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-09-11,182.470001,178.029999,178.100006,181.740005,5979400,181.740005
2017-09-12,185.330002,180.639999,182.550003,185.149994,6689600,185.149994
2017-09-13,184.500000,182.550003,184.070007,183.639999,4320300,183.639999
2017-09-14,185.289993,182.070007,183.250000,182.630005,4510700,182.630005
2017-09-15,184.929993,181.429993,182.729996,182.350006,5591700,182.350006
...,...,...,...,...,...,...
2020-06-24,472.359985,454.000000,468.540009,457.850006,4826200,457.850006
2020-06-25,467.010010,454.000000,458.859985,465.910004,4134500,465.910004
2020-06-26,468.029999,442.239990,466.390015,443.399994,6804700,443.399994


In [28]:
def ATR(DF, n):
    df = DF.copy()
    df["H-L"] = abs(df["High"] - df["Low"])
    df["H-PC"] = abs(df["High"] - df["Adj Close"].shift(1))
    df["L-PC"] = abs(df["Low"] - df["Adj Close"].shift(1))
    df["TR"] = df[["H-L","H-PC", "L-PC"]].max(axis=1, skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    df2 = df.drop(["H-L","H-PC", "L-PC"], axis=1)
    return df2 
    

In [29]:
atr = ATR(ohlcv, 20)